In [1]:
import os
import pandas as pd
import numpy as np

In [3]:
project_path = os.path.dirname(os.getcwd())
png = pd.read_csv(
    project_path + "/data/prices/outputs/2009/RTFP_country_2009_2022-08-17.csv")
png["date"] = pd.to_datetime(png["date"])

In [4]:
png

,Open,High,Low,Close,Inflation,country,ISO3,date
0,0.690540,0.742082,0.649058,0.695159,NaN,Papua New Guinea,PNG,2009-01-01
1,0.700222,0.747046,0.653399,0.709033,NaN,Papua New Guinea,PNG,2009-01-02
2,0.712256,0.764049,0.660462,0.716254,NaN,Papua New Guinea,PNG,2009-01-03
3,0.718400,0.773517,0.663282,0.716188,NaN,Papua New Guinea,PNG,2009-01-04
4,0.719868,0.776872,0.662863,0.722852,NaN,Papua New Guinea,PNG,2009-01-05
...,...,...,...,...,...,...,...,...
159,1.298422,1.385601,1.211244,1.255384,1.588927,Papua New Guinea,PNG,2022-01-04
160,1.274761,1.357783,1.191738,1.298791,8.031285,Papua New Guinea,PNG,2022-01-05
161,1.311183,1.397665,1.224701,1.296858,7.356945,Papua New Guinea,PNG,2022-01-06
162,1.308152,1.395347,1.220957,1.357078,12.759345,Papua New Guinea,PNG,2022-01-07


In [54]:
from bokeh.plotting import figure, show, output_file
from bokeh.models import ColumnDataSource, HoverTool, CrosshairTool, CDSView, BooleanFilter, BoxZoomTool

output_file("candlestick.html", title="candlestick.py example")


source = ColumnDataSource(data=png)
inc = png.Close > png.Open
inc = CDSView(source=source, filters=[BooleanFilter(inc)])
dec = png.Open > png.Close
dec = CDSView(source=source, filters=[BooleanFilter(dec)])
w = 24*60*60*1000  # day in ms

hover = HoverTool(
    tooltips=[
        ("date", "@date"),
        ("open", "@Open"),
        ("high", "@High"),
        ("close", "@Close"),
    ],
    formatters={
        '@date': 'datetime'
    },
)

TOOLS = [CrosshairTool(), hover, BoxZoomTool()]

p = figure(x_axis_type="datetime", tools=TOOLS)
#p.xaxis.major_label_orientation = pi/4
p.grid.grid_line_alpha = 0.3

p.segment(source=source, x0="date", y0="High",
          x1="date", y1="Low", color="black")
p.vbar(source=source, view=inc, width=w, x="datetime", top='Open', 
       bottom='Close', fill_color="green", line_color="green")
p.vbar(source=source, view=dec, width=w, x="datetime", top='Open', 
       bottom='Close', fill_color="red", line_color="red")


p.legend.location = "top_left"
p.legend.click_policy = "hide"
show(p)

/Users/czhang/miniforge3/lib/python3.9/site-packages/bokeh/models/plots.py:815: UserWarning:


You are attempting to set `plot.legend.location` on a plot that has zero legends added, this will have no effect.

Before legend properties can be set, you must add a Legend explicitly, or call a glyph method with a legend parameter set.


/Users/czhang/miniforge3/lib/python3.9/site-packages/bokeh/models/plots.py:815: UserWarning:


You are attempting to set `plot.legend.click_policy` on a plot that has zero legends added, this will have no effect.

Before legend properties can be set, you must add a Legend explicitly, or call a glyph method with a legend parameter set.


ERROR:bokeh.core.validation.check:E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name. This could either be due to a misspelling or typo, or due to an expected column being missing. : key "x" value "datetime" (closest match: "date") [renderer: GlyphRenderer(id='6750', ...)]
ERROR:bokeh.core.validation.check:E-1

In [11]:
import plotly.graph_objects as go

candlestick = go.Candlestick(
    x=png.date,
    open=png['Open'],
    high=png['High'],
    low=png['Low'],
    close=png['Close']
)

fig = go.Figure(data=[candlestick])
fig.update_layout(
    width=650, height=400,
    margin=dict(
        l=25, r=25, b=100, t=25, pad=2),
    title="Papua New Guinea",
    yaxis_title='Food Price',
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)',
)


fig.update_xaxes(showline=True, linewidth=1, linecolor='black')
fig.layout.font.family = 'Helvetica'

fig.show()
fig.write_html("candlestick.html")